In [525]:
import pandas as pd
import numpy as np
from trino.dbapi import connect 
from dateutil.relativedelta import relativedelta

In [526]:
# def read_from_hive(sql_script):
#     conn = connect(
#             host='presto.bstis.com',
#             port=8080,
#             user='hadoop',
#             catalog='hive',
#             #schema='default'
#             )
#     cur = conn.cursor()
#     cur.execute(sql_script)
# 
#     columns = [desc[0] for desc in cur.description]
#     rows = cur.fetchall()  # Fetch all rows at once
# 
#     df = pd.DataFrame(rows, columns=columns)
# 
#     cur.close()
#     conn.close()
# 
#     return df

### Claims Raw

In [527]:
# def fetch_monthly_data(start_date, end_date):
#     sSQL = f'''
#     select
#     division as tenantid, 
#     paiddate, 
#     claimtype, 
#     icd9_1_chapter, 
#    -- facility_indicator, 
#    -- revenue, 
#    -- cpt_code, 
#     servicecategory_details, 
#    -- provider_pcp_flag, 
#    -- providerspecialty, 
#    -- reversalstatus, 
#     personid, 
#     tpaclaimid, 
#     amtallowed, 
#     amtpaid, 
#    -- amtdeductible, 
#    -- amtcoins, 
#    -- amtcopay, 
#    -- amtcobpaid, 
#    -- amthra, 
#    -- amtpaidbyothersins, 
#     locationtype, 
#     locationtypedesc, 
#     therapeutic_class_name, 
#     tag_tpa, 
#     tag_exchange, 
#     claimid
# 
#     from hive.trinet.claims c
# 
#     where  paiddate <= date('{end_date}')
#      AND paiddate >= date('{start_date}')
#      and claimtype in ('MED', 'Rx')
# 
#     '''
# 
#     df_claims = read_from_hive(sSQL)
#     df_claims.to_csv(f'Claims Data/claims_raw_{start_date.replace("-", "_")}.gz',index=False, compression='gzip')
# 
# # Define the start and end dates
# start_date = pd.to_datetime('2023-04-01')
# end_date = pd.to_datetime('2023-04-30')
# 
# # Loop over each month
# while start_date <= end_date:
#     next_month = start_date + relativedelta(months=1)
#     fetch_monthly_data(start_date.strftime('%Y-%m-%d'), (next_month - pd.Timedelta(days=1)).strftime('%Y-%m-%d'))
#     start_date = next_month

In [528]:
# sSQL = '''
# select 
# claimid, inpatient_visitid, er_visitid, med_visitid
#  from hive.trinet.claims_inpatient
# where inpatient_visitid <> '' or er_visitid <> ''
# 
# '''
# 
# df_claims = read_from_hive(sSQL)
# del sSQL
# 
# df_claims.to_csv('Claims Data/claimsip_raw.gz',index=False, compression='gzip')

In [529]:
# def fetch_monthly_ccmap(start_date, end_date):
#     sSQL = f'''
#     select * 
#     from hive.trinet.ccmap
# 
#     where  date <= date('{end_date}')
#      AND date >= date('{start_date}')
#     '''
# 
#     df_ccmap = read_from_hive(sSQL)
#     df_ccmap.to_csv(f'Claims Data/ccmap_raw_{start_date.replace("-", "_")}.gz',index=False, compression='gzip')
# 
# # Define the start and end dates
# start_date = pd.to_datetime('2021-10-01')
# end_date = pd.to_datetime('2024-03-31')
# 
# # Loop over each month
# while start_date <= end_date:
#     next_month = start_date + relativedelta(months=1)
#     fetch_monthly_ccmap(start_date.strftime('%Y-%m-%d'), (next_month - pd.Timedelta(days=1)).strftime('%Y-%m-%d'))
#     start_date = next_month

### Set Period

In [530]:
def calculate_dates(input_date):
    curr_month = pd.to_datetime(input_date).strftime('%Y-%m-%d')
    stop_date = (pd.to_datetime(curr_month) + pd.DateOffset(months=1) - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    start_date = (pd.to_datetime(stop_date) - pd.DateOffset(months=12) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    return curr_month, stop_date, start_date

curr_month, stop_date, start_date = calculate_dates('2022-11-01')

In [531]:
def calculate_dates(curr_month):
    curr_month = pd.to_datetime(curr_month)
    stop_date_nonkaiser = (curr_month + pd.DateOffset(months=1) - pd.DateOffset(days=1))
    start_date_nonkaiser = (stop_date_nonkaiser - pd.DateOffset(months=12) + pd.DateOffset(days=1))
    stop_date_kaiser = (stop_date_nonkaiser - pd.DateOffset(months=1))
    start_date_kaiser = (start_date_nonkaiser - pd.DateOffset(months=1))
    
    return stop_date_nonkaiser, start_date_nonkaiser, stop_date_kaiser, start_date_kaiser

stop_date_nonkaiser, start_date_nonkaiser, stop_date_kaiser, start_date_kaiser = calculate_dates(curr_month)

### Pre-process Claims

In [ ]:
def read_data(date, is_kaiser):
    file_name = f'Claims Data/claims_raw_{date.strftime("%Y_%m_%d")}.gz'
    df = pd.read_csv(file_name, compression='gzip',
                     dtype={'tenantid':str,
                            'personid':str,
                            'claimtype':str,
                            'icd9_1_chapter':str,
                            'servicecategory_details':str,
                            'locationtype':str,
                            'locationtypedesc':str,
                            'therapeutic_class_name':str,
                            'tag_tpa':str,
                            'tag_exchange':str,
                            'claimid':str,
                            'amtpaid':np.float64,
                            'amtallowed':np.float64,
                            'tpaclaimid':str})
    if is_kaiser:
        return df[df['tag_tpa'] == 'kaiser']
    else:
        return df[df['tag_tpa'] != 'kaiser']

def read_claims_data(start_date, end_date, is_kaiser):
    dfs = []
    while start_date <= end_date:
        dfs.append(read_data(start_date, is_kaiser))
        start_date += relativedelta(months=1)
    df = pd.concat(dfs, axis=0)
    return df

df_claims_nonkaiser = read_claims_data(start_date_nonkaiser, stop_date_nonkaiser, False)
df_claims_kaiser = read_claims_data(start_date_kaiser, stop_date_kaiser, True)

In [ ]:
df_all_claims = pd.concat([df_claims_nonkaiser, df_claims_kaiser], axis=0)
del df_claims_nonkaiser, df_claims_kaiser

df_claims = df_all_claims.query("~((tag_tpa == 'bcbsmn' & tag_exchange == 'TriNet III') | "
                                "(tag_tpa == 'kaiser' & tag_exchange == 'TriNet IV') | "
                                "(tag_tpa == 'excellus' & tag_exchange == 'TriNet I'))")

### Catastrophic

In [ ]:
df_cat = (df_claims[['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'amtallowed', 'amtpaid', 'tpaclaimid']]
          .groupby(['tenantid', 'tag_tpa', 'tag_exchange', 'personid'])
          .agg({'amtallowed':'sum', 'amtpaid':'sum', 'tpaclaimid':'count'})
          .rename(columns={'tpaclaimid':'util'})
          .reset_index()
          .assign(**{'class':'Catastrophic'})
          .loc[lambda df: df['amtpaid'] >= 100000])

# df_cat

In [ ]:
df_cat_chapter = (df_claims[['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'icd9_1_chapter', 'amtallowed', 'amtpaid', 'tpaclaimid']]
                  .groupby(['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'icd9_1_chapter'])
                  .agg({'amtallowed':'sum', 'amtpaid':'sum', 'tpaclaimid':'nunique'})
                  .reset_index()
                  .rename(columns={'tpaclaimid':'claim_count'})
                  .merge(df_cat[['tenantid', 'tag_tpa', 'tag_exchange', 'personid']], on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='right'))
# df_cat_chapter

In [ ]:
df_cat_rx_chapter = (df_claims[['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'therapeutic_class_name', 'amtallowed', 'amtpaid', 'tpaclaimid']]
                     .groupby(['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'therapeutic_class_name'])
                     .agg({'amtallowed':'sum', 'amtpaid':'sum', 'tpaclaimid':'nunique'})
                     .reset_index()
                     .rename(columns={'tpaclaimid':'claim_count'})
                     .merge(df_cat[['tenantid', 'tag_tpa', 'tag_exchange', 'personid']], on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='right'))
# df_cat_rx_chapter

In [ ]:
df_cat_max = (df_cat_chapter.copy()
              .sort_values(by=['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'amtallowed'], ascending=[True,True, True, True,False])
              .groupby(['tenantid', 'tag_tpa', 'tag_exchange', 'personid'])
              .first()
              .reset_index())
# df_cat_max

In [ ]:
df_cat_rx_max = (df_cat_rx_chapter.copy()
                 .sort_values(by=['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'amtallowed'], ascending=[True,True, True, True,False])
                 .groupby(['tenantid', 'tag_tpa', 'tag_exchange', 'personid'])
                 .first()
                 .reset_index()
                 .assign(therapeutic_class_name = lambda df: df['therapeutic_class_name'].str.title()))
# df_cat_rx_max

In [ ]:
df_cat_icd9chapter = (df_cat_max.merge(df_cat_rx_max, how='outer', on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], suffixes=('_a', '_b'))
                      .assign(icd9_1_chapter = lambda df: df['icd9_1_chapter'].where(df['icd9_1_chapter'].notna(), df['therapeutic_class_name']))
                      .loc[:, ['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'icd9_1_chapter']]
                      .assign(**{'class':'Catastrophic'}))
# df_cat_icd9chapter

In [ ]:
df_cat_person = pd.merge(df_cat_icd9chapter, df_cat, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'class'], how='left')
# df_cat_person

In [ ]:
df_cat_groups = (df_cat_person.copy()
                 .loc[:, ['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'class', 'icd9_1_chapter']]
                 .rename(columns={'icd9_1_chapter':'group'}))
# df_cat_groups

In [ ]:
del df_cat_chapter, df_cat_rx_chapter, df_cat_max, df_cat_rx_max, df_cat_icd9chapter, df_cat_person

### Chronic

In [ ]:
df_carrier_nonkaiser = (df_claims[['tenantid', 'personid', 'tag_tpa', 'tag_exchange']]
                        .drop_duplicates()
                        .query("tag_tpa != 'kaiser'"))

df_carrier_kaiser = (df_claims[['tenantid', 'personid', 'tag_tpa', 'tag_exchange']]
                     .drop_duplicates()
                     .query("tag_tpa == 'kaiser'"))


In [ ]:
stop_date_nonkaiser = pd.to_datetime((pd.to_datetime(curr_month) + pd.DateOffset(months=1) - pd.DateOffset(days=1)).strftime('%Y-%m-%d'))
start_date_nonkaiser = pd.to_datetime((pd.to_datetime(stop_date_nonkaiser) - pd.DateOffset(months=12) + pd.DateOffset(days=1)).strftime('%Y-%m-%d'))
stop_date_kaiser = pd.to_datetime((pd.to_datetime(stop_date_nonkaiser) - pd.DateOffset(months=1)).strftime('%Y-%m-%d'))
start_date_kaiser = pd.to_datetime((pd.to_datetime(start_date_nonkaiser) - pd.DateOffset(months=1)).strftime('%Y-%m-%d'))

In [ ]:
def read_nonkaiser_ccmap(start_date_nonkaiser, stop_date_nonkaiser):
    file_name = f'Claims Data/ccmap_raw_{start_date_nonkaiser.replace("-", "_")}.gz'
    df_ccmap_nonkaiser = pd.read_csv(file_name, compression='gzip', dtype={'tenantid':str, 'personid':str})
    return df_ccmap_nonkaiser

# Initialize an empty DataFrame to hold all the data
df_ccmap_nonkaiser = pd.DataFrame()

# Loop over each month
while start_date_nonkaiser <= stop_date_nonkaiser:
    next_month = start_date_nonkaiser + relativedelta(months=1)
    df_monthly_ccmap_nonkaiser = read_nonkaiser_ccmap(start_date_nonkaiser.strftime('%Y-%m-%d'), (next_month - pd.Timedelta(days=1)).strftime('%Y-%m-%d'))
    df_ccmap_nonkaiser = pd.concat([df_ccmap_nonkaiser, df_monthly_ccmap_nonkaiser], axis=0)
    start_date_nonkaiser = next_month


df_ccmap_nonkaiser.drop(['tenantid'], axis=1, inplace=True)
df_ccmap_nonkaiser = pd.merge(df_carrier_nonkaiser, df_ccmap_nonkaiser, left_on=['tenantid','personid'], right_on=['division','personid'], how='left')
df_ccmap_nonkaiser.dropna(subset=['ccondition'], inplace=True)
# df_ccmap_nonkaiser

In [ ]:
def read_kaiser_ccmap(start_date_kaiser, stop_date_kaiser):
    file_name = f'Claims Data/ccmap_raw_{start_date_kaiser.replace("-", "_")}.gz'
    df_ccmap_kaiser = pd.read_csv(file_name, compression='gzip', dtype={'tenantid':str, 'personid':str})
    return df_ccmap_kaiser

# Initialize an empty DataFrame to hold all the data
df_ccmap_kaiser = pd.DataFrame()

# Loop over each month
while start_date_kaiser <= stop_date_kaiser:
    next_month = start_date_kaiser + relativedelta(months=1)
    df_monthly_ccmap_kaiser = read_kaiser_ccmap(start_date_kaiser.strftime('%Y-%m-%d'), (next_month - pd.Timedelta(days=1)).strftime('%Y-%m-%d'))
    df_ccmap_kaiser = pd.concat([df_ccmap_kaiser, df_monthly_ccmap_kaiser], axis=0)
    start_date_kaiser = next_month

df_ccmap_kaiser.drop(['tenantid'], axis=1, inplace=True)
# df_ccmap_kaiser.rename(columns={'division':'tenantid'})
df_ccmap_kaiser = pd.merge(df_carrier_kaiser, df_ccmap_kaiser, left_on=['tenantid','personid'], right_on=['division','personid'], how='left')
df_ccmap_kaiser.dropna(subset=['ccondition'], inplace=True)

In [ ]:
df_ccmap = pd.concat([df_ccmap_nonkaiser, df_ccmap_kaiser], axis=0)
del df_ccmap_nonkaiser, df_ccmap_kaiser
# df_ccmap

In [ ]:
def update_ccmap(df_ccmap, df_cat, condition_file):
    df_updated_condition = pd.read_csv(condition_file)
    df_ccmap = pd.merge(df_ccmap, df_cat[['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'amtallowed']],
                        on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')
    df_ccmap = df_ccmap[df_ccmap['amtallowed'].isna()]
    df_ccmap = pd.merge(df_ccmap, df_updated_condition, on=['ccondition_category'], how='left')
    return df_ccmap

df_updated_ccmap = update_ccmap(df_ccmap, df_cat, 'updated_condition.csv')

In [ ]:
df_ccmap_count = (df_updated_ccmap.groupby(['tenantid', 'tag_tpa', 'tag_exchange', 'personid'])
                  .agg({'updated_category':'nunique'})
                  .rename(columns={'updated_category':'condition_count'})
                  .reset_index())

# df_ccmap_count

In [ ]:
# Filter for 'cancer', select columns, and drop duplicates
df_ccmap_cancer_all = (df_updated_ccmap[df_updated_ccmap['updated_category'].str.lower() == 'cancer']
                       [['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
                       .drop_duplicates())

# Merge with df_ccmap_count
df_ccmap_cancer_all = df_ccmap_cancer_all.merge(df_ccmap_count, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')

# Create 'cancer only' and 'cancer complex' groups
df_ccmap_cancer_only = df_ccmap_cancer_all.loc[df_ccmap_cancer_all['condition_count'] == 1].assign(group='cancer only')
df_ccmap_cancer_comorb = df_ccmap_cancer_all.loc[df_ccmap_cancer_all['condition_count'] != 1].assign(group='cancer complex')

# df_ccmap_cancer_all

In [ ]:
# Filter for 'heart disease', select columns, and drop duplicates
df_ccmap_cardiac_all = (df_updated_ccmap[df_updated_ccmap['updated_category'].str.lower() == 'heart disease']
                        [['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
                        .drop_duplicates())

# Merge with df_ccmap_cancer_all and df_ccmap_count, and filter rows
df_ccmap_cardiac_all = (df_ccmap_cardiac_all
                        .merge(df_ccmap_cancer_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')
                        .loc[lambda df: df['condition_count'].isna()]
                        .drop('condition_count', axis=1)
                        .merge(df_ccmap_count, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left'))

# Create 'heart disease only' and 'heart disease complex' groups
df_ccmap_cardiac_only = df_ccmap_cardiac_all.loc[df_ccmap_cardiac_all['condition_count'] == 1].assign(group='heart disease only')
df_ccmap_cardiac_comorb = df_ccmap_cardiac_all.loc[df_ccmap_cardiac_all['condition_count'] != 1].assign(group='heart disease complex')

# df_ccmap_cardiac_all

In [ ]:
# Filter for 'chronic kidney disease', select columns, and drop duplicates
df_ccmap_ckd_all = (df_updated_ccmap[df_updated_ccmap['updated_category'].str.lower() == 'chronic kidney disease']
                    [['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
                    .drop_duplicates())

# Merge with df_ccmap_cancer_all and df_ccmap_cardiac_all, and filter rows
df_ccmap_ckd_all = (df_ccmap_ckd_all
                    .merge(df_ccmap_cancer_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_a', '_b'))
                    .merge(df_ccmap_cardiac_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_c', '_d'))
                    .loc[lambda df: df.filter(like='condition_count').isna().all(axis=1)])

# Drop columns that contain 'condition_count' in their names
df_ccmap_ckd_all = df_ccmap_ckd_all.drop(df_ccmap_ckd_all.filter(like='condition_count').columns, axis=1)

# Merge with df_ccmap_count
df_ccmap_ckd_all = df_ccmap_ckd_all.merge(df_ccmap_count, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')

# Create 'CKD only' and 'CKD complex' groups
df_ccmap_ckd_only = df_ccmap_ckd_all.loc[df_ccmap_ckd_all['condition_count'] == 1].assign(group='CKD only')
df_ccmap_ckd_comorb = df_ccmap_ckd_all.loc[df_ccmap_ckd_all['condition_count'] != 1].assign(group='CKD complex')

# df_ccmap_ckd_all

In [ ]:
# Filter for 'lung disease', select columns, and drop duplicates
df_ccmap_lung_all = (df_updated_ccmap[df_updated_ccmap['updated_category'].str.lower() == 'lung disease']
                     [['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
                     .drop_duplicates())

# Merge with df_ccmap_cancer_all, df_ccmap_cardiac_all, and df_ccmap_ckd_all, and filter rows
df_ccmap_lung_all = (df_ccmap_lung_all
                     .merge(df_ccmap_cancer_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_a', '_b'))
                     .merge(df_ccmap_cardiac_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_c', '_d'))
                     .merge(df_ccmap_ckd_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_e', '_f'))
                     .loc[lambda df: df.filter(like='condition_count').isna().all(axis=1)])

# Drop columns that contain 'condition_count' in their names
df_ccmap_lung_all = df_ccmap_lung_all.drop(df_ccmap_lung_all.filter(like='condition_count').columns, axis=1)

# Merge with df_ccmap_count
df_ccmap_lung_all = df_ccmap_lung_all.merge(df_ccmap_count, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')

# Create 'lung disease only' and 'lung disease complex' groups
df_ccmap_lung_only = df_ccmap_lung_all.loc[df_ccmap_lung_all['condition_count'] == 1].assign(group='lung disease only')
df_ccmap_lung_comorb = df_ccmap_lung_all.loc[df_ccmap_lung_all['condition_count'] != 1].assign(group='lung disease complex')
# df_ccmap_lung_all

In [ ]:
# Filter for 'mental, behavioral and neurological', select columns, and drop duplicates
df_ccmap_mental_all = (df_updated_ccmap[df_updated_ccmap['updated_category'].str.lower() == 'mental, behavioral and neurological']
                       [['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
                       .drop_duplicates())

# Merge with df_ccmap_cancer_all, df_ccmap_cardiac_all, df_ccmap_ckd_all, and df_ccmap_lung_all, and filter rows
df_ccmap_mental_all = (df_ccmap_mental_all
                       .merge(df_ccmap_cancer_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_a', '_b'))
                       .merge(df_ccmap_cardiac_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_c', '_d'))
                       .merge(df_ccmap_ckd_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_e', '_f'))
                       .merge(df_ccmap_lung_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_g', '_h'))
                       .loc[lambda df: df.filter(like='condition_count').isna().all(axis=1)])

# Drop columns that contain 'updated_category' in their names
df_ccmap_mental_all = df_ccmap_mental_all.drop(df_ccmap_mental_all.filter(like='condition_count').columns, axis=1)

# Merge with df_ccmap_count
df_ccmap_mental_all = df_ccmap_mental_all.merge(df_ccmap_count, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')

# Create 'mental health only' and 'mental health complex' groups
df_ccmap_mental_only = df_ccmap_mental_all.loc[df_ccmap_mental_all['condition_count'] == 1].assign(group='mental health only')
df_ccmap_mental_comorb = df_ccmap_mental_all.loc[df_ccmap_mental_all['condition_count'] != 1].assign(group='mental health complex')
# df_ccmap_mental_all


In [ ]:
# Filter for 'diabetes', select columns, drop duplicates, and merge with other DataFrames
df_ccmap_diabetes_all = (df_updated_ccmap[df_updated_ccmap['updated_category'].str.lower() == 'diabetes']
                         [['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
                         .drop_duplicates())
# Merge with df_ccmap_cancer_all, df_ccmap_cardiac_all, df_ccmap_ckd_all, df_ccmap_lung_all, and df_ccmap_mental_all, and filter rows
df_ccmap_diabetes_all = (df_ccmap_diabetes_all
                         .merge(df_ccmap_cancer_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_a', '_b'))
                         .merge(df_ccmap_cardiac_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_c', '_d'))
                         .merge(df_ccmap_ckd_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_e', '_f'))
                         .merge(df_ccmap_lung_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_g', '_h'))
                         .merge(df_ccmap_mental_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_i', '_j'))
                         .loc[lambda df: df.filter(like='condition_count').isna().all(axis=1)])

# Drop columns that contain 'condition_count' in their names
df_ccmap_diabetes_all = df_ccmap_diabetes_all.drop(df_ccmap_diabetes_all.filter(like='condition_count').columns, axis=1)

# Merge with df_ccmap_count
df_ccmap_diabetes_all = df_ccmap_diabetes_all.merge(df_ccmap_count, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')

# Create 'diabetes only' and 'diabetes complex' groups
df_ccmap_diabetes_only = df_ccmap_diabetes_all.loc[df_ccmap_diabetes_all['condition_count'] == 1].assign(group='diabetes only')
df_ccmap_diabetes_comorb = df_ccmap_diabetes_all.loc[df_ccmap_diabetes_all['condition_count'] != 1].assign(group='diabetes complex')
# df_ccmap_diabetes_all

In [ ]:
# Filter for 'metabolic', select columns, drop duplicates, and merge with other DataFrames
df_ccmap_metabolic_all = (df_updated_ccmap[df_updated_ccmap['updated_category'].str.lower() == 'endocrine, nutritional, digestive and metabolic']
                         [['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
                         .drop_duplicates())

# Merge with df_ccmap_cancer_all, df_ccmap_cardiac_all, df_ccmap_ckd_all, df_ccmap_lung_all, df_ccmap_mental_all, and df_ccmap_diabetes_all, and filter rows
df_ccmap_metabolic_all = (df_ccmap_metabolic_all
                         .merge(df_ccmap_cancer_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_a', '_b'))
                         .merge(df_ccmap_cardiac_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_c', '_d'))
                         .merge(df_ccmap_ckd_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_e', '_f'))
                         .merge(df_ccmap_lung_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_g', '_h'))
                         .merge(df_ccmap_mental_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_i', '_j'))
                         .merge(df_ccmap_diabetes_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_k', '_l'))
                         .loc[lambda df: df.filter(like='condition_count').isna().all(axis=1)])

# Drop columns that contain 'condition_count' in their names
df_ccmap_metabolic_all = df_ccmap_metabolic_all.drop(df_ccmap_metabolic_all.filter(like='condition_count').columns, axis=1)

# Merge with df_ccmap_count
df_ccmap_metabolic_all = df_ccmap_metabolic_all.merge(df_ccmap_count, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')

# Create 'metabolic only' and 'metabolic complex' groups
df_ccmap_metabolic_only = df_ccmap_metabolic_all.loc[df_ccmap_metabolic_all['condition_count'] == 1].assign(group='metabolic only')
df_ccmap_metabolic_comorb = df_ccmap_metabolic_all.loc[df_ccmap_metabolic_all['condition_count'] != 1].assign(group='metabolic complex')
# df_ccmap_metabolic_all

In [ ]:
# Filter for 'metabolic', select columns, drop duplicates, and merge with other DataFrames
df_ccmap_MSK_all = (df_updated_ccmap[df_updated_ccmap['updated_category'].str.lower() == 'musculoskeletal and injury']
                         [['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
                         .drop_duplicates())

# Merge with df_ccmap_cancer_all, df_ccmap_cardiac_all, df_ccmap_ckd_all, df_ccmap_lung_all, df_ccmap_mental_all, df_ccmap_diabetes_all, and df_ccmap_metabolic_all, and filter rows
df_ccmap_MSK_all = (df_ccmap_MSK_all
                         .merge(df_ccmap_cancer_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_a', '_b'))
                         .merge(df_ccmap_cardiac_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_c', '_d'))
                         .merge(df_ccmap_ckd_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_e', '_f'))
                         .merge(df_ccmap_lung_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_g', '_h'))
                         .merge(df_ccmap_mental_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_i', '_j'))
                         .merge(df_ccmap_diabetes_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_k', '_l'))
                         .merge(df_ccmap_metabolic_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_m', '_n'))
                         .loc[lambda df: df.filter(like='condition_count').isna().all(axis=1)])

# Drop columns that contain 'condition_count' in their names
df_ccmap_MSK_all = df_ccmap_MSK_all.drop(df_ccmap_MSK_all.filter(like='condition_count').columns, axis=1)

# Merge with df_ccmap_count
df_ccmap_MSK_all = df_ccmap_MSK_all.merge(df_ccmap_count, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')

# Create 'metabolic only' and 'metabolic complex' groups
df_ccmap_MSK_only = df_ccmap_MSK_all.loc[df_ccmap_MSK_all['condition_count'] == 1].assign(group='MSK only')
df_ccmap_MSK_comorb = df_ccmap_MSK_all.loc[df_ccmap_MSK_all['condition_count'] != 1].assign(group='MSK complex')
# df_ccmap_MSK_all

In [ ]:
df_ccmap_other_all = (df_updated_ccmap
                         [['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
                        .drop_duplicates())

# Merge with df_ccmap_cancer_all, df_ccmap_cardiac_all, df_ccmap_ckd_all, df_ccmap_lung_all, df_ccmap_mental_all, df_ccmap_diabetes_all, and df_ccmap_metabolic_all, and filter rows
df_ccmap_other_all = (df_ccmap_other_all
                         .merge(df_ccmap_cancer_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_a', '_b'))
                         .merge(df_ccmap_cardiac_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_c', '_d'))
                         .merge(df_ccmap_ckd_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_e', '_f'))
                         .merge(df_ccmap_lung_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_g', '_h'))
                         .merge(df_ccmap_mental_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_i', '_j'))
                         .merge(df_ccmap_diabetes_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_k', '_l'))
                         .merge(df_ccmap_metabolic_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_m', '_n'))
                         .merge(df_ccmap_MSK_all, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left', suffixes=('_o', '_p'))
                         .loc[lambda df: df.filter(like='condition_count').isna().all(axis=1)])

# Drop columns that contain 'condition_count' in their names
df_ccmap_other_all = df_ccmap_other_all.drop(df_ccmap_other_all.filter(like='condition_count').columns, axis=1)

# Merge with df_ccmap_count
df_ccmap_other_all = df_ccmap_other_all.merge(df_ccmap_count, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')

# Create 'other single' and 'other complex' groups
df_ccmap_other_only = df_ccmap_other_all.loc[df_ccmap_other_all['condition_count'] == 1].assign(group='other single')
df_ccmap_other_comorb = df_ccmap_other_all.loc[df_ccmap_other_all['condition_count'] != 1].assign(group='other complex')

# df_ccmap_other_all

In [ ]:
df_ccgroups = pd.concat([df_ccmap_cancer_only, df_ccmap_cancer_comorb, df_ccmap_cardiac_only, df_ccmap_cardiac_comorb, df_ccmap_ckd_only, df_ccmap_ckd_comorb, df_ccmap_lung_only, df_ccmap_lung_comorb, df_ccmap_mental_only, df_ccmap_mental_comorb, df_ccmap_diabetes_only, df_ccmap_diabetes_comorb, df_ccmap_metabolic_only, df_ccmap_metabolic_comorb, df_ccmap_MSK_only, df_ccmap_MSK_comorb, df_ccmap_other_only, df_ccmap_other_comorb], axis=0)

del df_ccmap_cancer_only, df_ccmap_cancer_comorb, df_ccmap_cardiac_only, df_ccmap_cardiac_comorb, df_ccmap_ckd_only, df_ccmap_ckd_comorb, df_ccmap_lung_only, df_ccmap_lung_comorb, df_ccmap_mental_only, df_ccmap_mental_comorb, df_ccmap_diabetes_only, df_ccmap_diabetes_comorb, df_ccmap_metabolic_only, df_ccmap_metabolic_comorb, df_ccmap_MSK_only, df_ccmap_MSK_comorb, df_ccmap_other_only, df_ccmap_other_comorb

df_ccgroups['class'] = 'Chronic Condition'
# df_ccgroups

### Acute

In [ ]:
# Load the data
df_claimsip = pd.read_csv('Claims Data/claimsip_raw.gz', compression='gzip', dtype={'claimid':str})

# Create df_acute_er DataFrame
df_acute_er = (
    df_claims[['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'claimid']]
    .merge(df_claimsip, on=['claimid'], how='left')
    .query("`er_visitid`.notna()")
    .loc[:, ['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
    .drop_duplicates()
    .merge(df_cat, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')
    .query("`amtallowed`.isna()")
    .merge(df_updated_ccmap, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')
    .query("`updated_category`.isna()")
    .loc[:, ['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
    .drop_duplicates()
    .assign(**{'class':'Acute'}, group='acute_ER')
)


# df_acute_er

In [ ]:
df_acute_noner = (
    df_claims[['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'claimid']]
    .merge(df_claimsip, on=['claimid'], how='left')
    .query("`er_visitid`.isna()")
    .merge(df_acute_er, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')
    .query("`group`.isna()")
    .loc[:, ['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
    .drop_duplicates()
    .merge(df_cat, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')
    .query("`amtallowed`.isna()")
    .merge(df_updated_ccmap, on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'], how='left')
    .query("`updated_category`.isna()")
    .loc[:, ['tenantid', 'tag_tpa', 'tag_exchange', 'personid']]
    .drop_duplicates()
    .assign(**{'class':'Acute'}, group='acute_NonER')
)

# del df_acute
# df_acute_noner = df_acute_noner[df_acute_noner['tenantid'] == 'VEQ-Nutcracker Therapeutics, Inc']
# df_acute_noner

In [ ]:
df_personid_groups = pd.concat([df_cat_groups, df_ccgroups, df_acute_er, df_acute_noner], axis=0)
# df_personid_groups

In [ ]:
del df_cat_groups, df_ccgroups, df_acute_er, df_acute_noner

In [ ]:
# Copy df_claims to df_dmca_groups and merge with df_claimsip
df_dmca_groups = df_claims.copy().merge(df_claimsip, on=['claimid'], how='left')

# Define the conditions
conditions = [
    (df_dmca_groups['claimtype'] == 'MED') & (df_dmca_groups['inpatient_visitid'].notna() | df_dmca_groups['servicecategory_details'].str.lower().str.contains('inpatient')),
    (df_dmca_groups['claimtype'] == 'MED') & df_dmca_groups['er_visitid'].notna(),
    (df_dmca_groups['claimtype'] == 'MED') & df_dmca_groups['servicecategory_details'].str.lower().str.contains('outpatient') & ~df_dmca_groups['servicecategory_details'].str.lower().isin(['outpatient - office', 'outpatient - mh/sa']) & df_dmca_groups['inpatient_visitid'].isna() & df_dmca_groups['er_visitid'].isna(),
    (df_dmca_groups['claimtype'] == 'MED') & df_dmca_groups['servicecategory_details'].str.lower().str.contains('outpatient') & df_dmca_groups['servicecategory_details'].str.lower().isin(['outpatient - office', 'outpatient - mh/sa']) & df_dmca_groups['inpatient_visitid'].isna() & df_dmca_groups['er_visitid'].isna(),
    (df_dmca_groups['claimtype'] == 'Rx')
]

# Define the outputs
outputs = ['IP', 'ER', 'Outpatient', 'Office Visit', 'Rx']

# Create 'service_type' column
df_dmca_groups['service_type'] = np.select(conditions, outputs, default=np.nan)

# Merge with df_personid_groups and select necessary columns
df_dmca_groups = df_dmca_groups.merge(df_personid_groups, how='inner', on=['tenantid', 'tag_tpa', 'tag_exchange', 'personid'])
df_dmca_groups = df_dmca_groups[['tenantid', 'tag_tpa', 'tag_exchange', 'personid', 'class', 'group', 'service_type', 'inpatient_visitid', 'er_visitid', 'tpaclaimid', 'amtallowed', 'amtpaid']]

# Filter out 'nan' and create 'counts_field' column
df_dmca_groups = df_dmca_groups[df_dmca_groups['service_type'] != 'nan']
df_dmca_groups['counts_field'] = df_dmca_groups.apply(lambda row: row['inpatient_visitid'] if row['service_type'] == 'IP' else (row['er_visitid'] if row['service_type'] == 'ER' else row['tpaclaimid']), axis=1)

# Group by 'tenantid', 'tag_tpa', 'tag_exchange', 'class', 'group' and calculate unique 'personid'
df_claimants = df_dmca_groups.groupby(['tenantid', 'tag_tpa', 'tag_exchange', 'class', 'group'])['personid'].nunique().reset_index()

# Group by 'tenantid', 'tag_tpa', 'tag_exchange', 'class', 'group', 'service_type' and calculate necessary values
df_dmca_groups = df_dmca_groups.groupby(['tenantid', 'tag_tpa', 'tag_exchange', 'class', 'group', 'service_type']).agg({'counts_field':'nunique', 'amtallowed':'sum', 'amtpaid':'sum'}).reset_index()

# Merge the two DataFrames
df_dmca_groups = pd.merge(df_dmca_groups, df_claimants, on=['tenantid', 'tag_tpa', 'tag_exchange', 'class', 'group'])

# Rename columns
df_dmca_groups.rename(columns={'counts_field':'counts', 'amtallowed': 'allowed', 'amtpaid': 'paid', 'personid': 'claimants'}, inplace=True)

# Add 'year', 'start_date', and 'stop_date' columns
df_dmca_groups['year'] = pd.to_datetime(start_date).strftime('%Y')
df_dmca_groups['start_date'] = start_date
df_dmca_groups['stop_date'] = stop_date

# Save to CSV file
df_dmca_groups.to_csv(f'Claims Data/claims_{curr_month.replace("-", "_")}.csv',index=False)

# df_dmca_groups